<a href="https://colab.research.google.com/github/Keveen-ghori/Question-Answering-System-using-Haystack/blob/main/Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Tue May 17 07:00:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Haystack Setup

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-9ru9qk5c/farm-haystack_2ea3dbca602649baa9608177f81c6637
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-9ru9qk5c/farm-haystack_2ea3dbca602649baa9608177f81c6637
  Resolved https://github.com/deepset-ai/haystack.git to commit 686e9d24ef3b7cc2fefe904a0857904de38d6962
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 72.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0
    Uninstalling torch-1.9.0:
      Successfully uninstalled torch-1.9.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0
    Uninstalling torchvision-0.10.0:
      Successfully uninst

## import required Dependencies

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader, PDFToTextConverter

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


## Start Elasticsearch using Docker

In [ ]:
# Recommended: Start Elasticsearch using Docker via the Haystack utility function
from haystack.utils import launch_es

launch_es()

WARNING - haystack.utils.doc_store -  Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 


## Setup Elasticsearch Version 7.9.2

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
import requests 
import json

## Requests the existing clusters in elasticsearch

In [ ]:
requests.get('http://localhost:9200/_cluster/health').json()

{'active_primary_shards': 2,
 'active_shards': 2,
 'active_shards_percent_as_number': 50.0,
 'cluster_name': 'elasticsearch',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 1,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 1,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'yellow',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 2}

In [ ]:
print(requests.get('http://localhost:9200/_cat/indices').text)

yellow open aurelius _CiIYQr2SZWith2AV7imCw 1 1 848 665 152.8kb 152.8kb
yellow open label    1QCsHFJhRSKUZttsDol78w 1 1   0   0    208b    208b



In [ ]:
# Connect to Elasticsearch

from haystack.document_stores import ElasticsearchDocumentStore

doc_store = ElasticsearchDocumentStore(
    host="localhost", 
    username="", 
    password="", 
    index="aurelius")

In [ ]:
print(requests.get('http://localhost:9200/_cat/indices').text)

yellow open aurelius _CiIYQr2SZWith2AV7imCw 1 1 848 665 152.8kb 152.8kb
yellow open label    1QCsHFJhRSKUZttsDol78w 1 1   0   0    208b    208b



In [ ]:
print(requests.get('http://localhost:9200/aurelius').json())

{'aurelius': {'aliases': {}, 'mappings': {'dynamic_templates': [{'strings': {'path_match': '*', 'match_mapping_type': 'string', 'mapping': {'type': 'keyword'}}}], 'properties': {'content': {'type': 'text'}, 'content_type': {'type': 'keyword'}, 'embedding': {'type': 'dense_vector', 'dims': 768}, 'name': {'type': 'keyword'}, 'source': {'type': 'keyword'}}}, 'settings': {'index': {'number_of_shards': '1', 'provided_name': 'aurelius', 'creation_date': '1652770240146', 'analysis': {'analyzer': {'default': {'type': 'standard'}}}, 'number_of_replicas': '1', 'uuid': '_CiIYQr2SZWith2AV7imCw', 'version': {'created': '7090299'}}}}}


## Collecting & Pre-processing Data

In [ ]:
res = requests.get('https://www.ietf.org/archive/id/draft-foudil-securitytxt-12.txt')

In [ ]:
orgdata = res.text
orgdata = orgdata.split('Translated by George Long')[0]

In [ ]:
#importing RegEx
import re
orgdata = re.sub('-{2,}', '', orgdata)
orgdata = re.sub('BOOK [A-Z]+\n', '', orgdata)

## Print Existing Data

In [ ]:
print(orgdata)





Network Working Group                                          E. Foudil
Internet-Draft                                                          
Intended status: Informational                           Y. Shafranovich
Expires: 25 November 2021                       Nightwatch Cybersecurity
                                                             24 May 2021


       A File Format to Aid in Security Vulnerability Disclosure
                      draft-foudil-securitytxt-12

Abstract

   When security vulnerabilities are discovered by researchers, proper
   reporting channels are often lacking.  As a result, vulnerabilities
   may be left unreported.  This document defines a machine-parsable
   format ("security.txt") to help organizations describe their
   vulnerability disclosure practices to make it easier for researchers
   to report vulnerabilities.

Status of This Memo

   This Internet-Draft is submitted in full conformance with the
   provisions of BCP 78 and BCP 79.

  

In [ ]:
with open('/content/data/meditations.txt', 'w') as f:
    for sentence in orgdata:
        f.write(sentence+ '')

In [ ]:
with open('/content/data/meditations.txt', 'r') as f:
    data = f.read()

In [ ]:
print(data[:500])





Network Working Group                                          E. Foudil
Internet-Draft                                                          
Intended status: Informational                           Y. Shafranovich
Expires: 25 November 2021                       Nightwatch Cybersecurity
                                                             24 May 2021


       A File Format to Aid in Security Vulnerability Disclosure
                      draft-foudil-securitytxt-12

Abstract

   


In [ ]:
data = data.split('\n')

In [ ]:
len(data)

1513

## General formate of JSON qa

In [ ]:
data_json = [
    {
        'content': paragraph,
        'meta': {
            'source': 'https://www.ietf.org'
        }
    } for paragraph in data
]

In [ ]:
len(data_json)

1513

In [ ]:
doc_store.write_documents(data_json)

In [ ]:
requests.get('http://localhost:9200/aurelius/_count').json()

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'count': 848}

## Retrive Process using PIPELINE

In [ ]:
from haystack.retriever.dense import DensePassageRetriever

## Questin Answering BERT Model (seq2seq)

In [ ]:
# facebook/dpr-question_encoder-single-nq-base HuggingFace Transformers Model for query Encoding
# facebook/dpr-ctx_encoder-single-nq-base HuggingFace Transformer Model for passage emboding
retriever = DensePassageRetriever(
    document_store=doc_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-question_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-question_encoder-single-nq-base
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  ===

## RETRIVER

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(doc_store)

## READER

In [ ]:

# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True,
                    context_window_size=1500)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


## Haystack PIPELINE

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader=reader, retriever=retriever)

## Execute Query

In [ ]:
# The higher top_k_retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="in which formate document will be defined?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 51.44 Batches/s]


In [ ]:
print_answers(prediction, details="minimum")



Query: in which formate document will be defined?
Answers:
[   {   'answer': 'BCP',
        'context': '   "OPTIONAL" in this document are to be interpreted as '
                   'described in BCP'},
    {   'answer': 'security.txt',
        'context': '   in "security.txt" files, defined by this '
                   'specification.'},
    {   'answer': 'section 4 of [RFC2142',
        'context': '   conventions defined in section 4 of [RFC2142].'},
    {   'answer': 'New or updated status',
        'context': '   5.  New or updated status, which MUST be one of:'},
    {   'answer': 'text file',
        'context': '   This document defines a text file to be placed in a '
                   'known location'}]


## FASTAPI

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install uvicorn 

In [ ]:
%matplotlib inline

In [ ]:
!pip install Flask==2.0.1 torchvision==0.10.0

  Using cached torchvision-0.10.0-cp37-cp37m-manylinux1_x86_64.whl (22.1 MB)
  Using cached torch-1.9.0-cp37-cp37m-manylinux1_x86_64.whl (831.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalled torchvision-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.
farm-haystack 1.4.1rc0 requires torch<1.12,>1.9, but you have torch 1.9.0 which is incompatible.


In [ ]:
from flask import Flask
APP = Flask(__name__)

@APP.route('/query', methods=['GET'])
async def get_query(q: str, retriever_limit: int = 10, reader_limit: int = 3):
    # get answers
    return pipe.run(query=q,
                        top_k_retriever=retriever_limit,
                        top_k_reader=reader_limit)


APP.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO - werkzeug -   * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)


In [ ]:
# import requests, json
# requests.get('http://127.0.0.1:5000/query')